# Aim : Code to convert pdf to text

So there are a number of libraries that can be used to convert pdf to text.

1. pymupdf4llm

Let's go through each of them.

## Loading our pdf files

In [1]:
# Let's load all the pdf files from the directory

import os
import dotenv

dotenv.load_dotenv()

folder_path = os.getenv("CLAIM_LOCATION")

pdf_files = [folder_path + "/" + f for f in os.listdir(folder_path) if f.endswith(".pdf")]

print(pdf_files)

['/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Compliance Report 4.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Compliance Report 1.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Compliance Report 2.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Compliance Report 3.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Sleep Study Report 3.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Sleep Study Report 2.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Sleep Study Report 1.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Sleep Study Report 4.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Order 3.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Delivery Ticket 2.pdf', '/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Physician Notes 1.pdf', '/Users/deveshsurve/UNIVERSITY/P

# First, let's try pymupdf4llm

In [2]:
def extract_text_pymupdf4llm(pdf_file):
    import pymupdf4llm
    md_text = pymupdf4llm.to_markdown(pdf_file)
    return md_text


In [3]:
md_text = extract_text_pymupdf4llm(pdf_files[1])
print(md_text)


Processing /Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Compliance Report 1.pdf...
[                                        ] (0/4=========[==========                              ] (1/4=========[====================                    ] (2/4=========[==============================          ] (3/4=========[========================================] (4/4]
FORT LAUDERDALE COMPL
6600 NW 12TH AVE STE 217
FORT LAUDERDALE
Florida, 33309

Phone: 954-772-5052
Email: LSOLOGUREN@AHCEQUIP.COM


##### ABREU, JUAN


# Compliance Report


##### 07/19/2023 - 08/17/2023

Patient ID: 113538

DOB: 08/12/1978

Age: 45 years

Gender: Male


#### Initial compliance period 07/19/2023 - 08/17/2023

Compliance met Yes

Compliance percentage 100%

Payor Medicare

#### Usage 07/19/2023 - 08/17/2023

Usage days 30/30 days (100%)

>= 4 hours 30 days (100%)

< 4 hours 0 days (0%)

Usage hours 323 hours 21 minutes

Average usage (total days) 10 hours 47 minutes

Average usage (days used) 10 hours 47

# Next, let's try pymupdf


In [4]:
def extract_text_pymupdf(pdf_file):
  import pymupdf # imports the pymupdf library

  total_text = ""

  doc = pymupdf.open(pdf_file) # open a document
  for page in doc: # iterate the document pages
        text = page.get_text() # get plain text encoded as UTF-8
        total_text += text
  return total_text

total_text = extract_text_pymupdf(pdf_files[1])

In [5]:
print(total_text)

ABREU, JUAN
07/19/2023 - 08/17/2023
Patient ID: 113538
DOB: 08/12/1978
Age: 45 years
Gender: Male
FORT LAUDERDALE COMPL
6600 NW 12TH AVE STE 217
FORT LAUDERDALE
Florida, 33309
Phone: 954-772-5052
Email: LSOLOGUREN@AHCEQUIP.COM
Compliance Report
Initial compliance period
07/19/2023 - 08/17/2023
Compliance met
Yes
Compliance percentage
100%
Payor
Medicare
Usage
07/19/2023 - 08/17/2023
Usage days
30/30 days (100%)
>= 4 hours
30 days (100%)
< 4 hours
0 days (0%)
Usage hours
323 hours 21 minutes
Average usage (total days)
10 hours 47 minutes
Average usage (days used)
10 hours 47 minutes
Median usage (days used)
11 hours 14 minutes
Total used hours (value since last reset - 08/17/2023)
931 hours
AirSense 11 AutoSet
Serial number
22231038074
Mode
CPAP
Set pressure
15 cmH2O
EPR
Fulltime
EPR level
3
Therapy
Leaks - L/min
Median: 120.0
95th percentile:120.0
Maximum:120.0
Events per hour
AI:
8.7
HI:
0.4
AHI:
9.1
Apnea Index
Central:
0.0
Obstructive:
0.0
Unknown:
8.6
RERA Index
1.0
Cheyne-Stokes r

# Next, let's try pdfminer-six

In [6]:
def extract_text_pdfminer(pdf_file):    
    from pdfminer.high_level import extract_text
    text = extract_text(pdf_file)
    return text

text = extract_text_pdfminer(pdf_files[1])


In [7]:
print(text)

FORT LAUDERDALE COMPL
6600 NW 12TH AVE STE 217
FORT LAUDERDALE
Florida, 33309

Phone: 954-772-5052
Email: LSOLOGUREN@AHCEQUIP.COM

Compliance Report

ABREU, JUAN

07/19/2023 - 08/17/2023

Patient ID: 113538
DOB: 08/12/1978
Age: 45 years
Gender: Male

Initial compliance period

07/19/2023 - 08/17/2023

Compliance met

Compliance percentage

Payor

Usage

Usage days

>= 4 hours

< 4 hours

Usage hours

Average usage (total days)

Average usage (days used)

Median usage (days used)

Yes

100%

Medicare

07/19/2023 - 08/17/2023

30/30 days (100%)

30 days (100%)

0 days (0%)

323 hours 21 minutes

10 hours 47 minutes

10 hours 47 minutes

11 hours 14 minutes

Total used hours (value since last reset - 08/17/2023)

931 hours

AirSense 11 AutoSet

Serial number

Mode

Set pressure

EPR

EPR level

Therapy

22231038074

CPAP

15 cmH2O

Fulltime

3

Leaks - L/min

Median: 120.0

95th percentile:120.0

Maximum:120.0

Events per hour

AI:

8.7

HI:

0.4

AHI:

9.1

Apnea Index

Central:

0.0

Ob

# Next, let's try llama-index


In [8]:
def extract_text_llama_index(pdf_file):
    from llama_parse import LlamaParse
    from llama_index.core import SimpleDirectoryReader
    import nest_asyncio; nest_asyncio.apply()

    # set up parser
    parser = LlamaParse(
        result_type="markdown"  # "markdown" and "text" are available
    )

    # use SimpleDirectoryReader to parse our file
    file_extractor = {".pdf": parser}
    documents = SimpleDirectoryReader(input_files=[pdf_file], file_extractor=file_extractor).load_data()
    return documents[0].text

text = extract_text_llama_index(pdf_files[1])


Started parsing the file under job_id 22db9d02-6729-412f-83ad-626c9d69b266


In [9]:
print(text)

# Compliance Report

Patient Name: Abreu, Juan

Address: 6600 NW 12TH AVE STE 217, Fort Lauderdale, Florida, 33309

Phone: 954-772-5052

Email: LSOLOGUREN@AHCEQUIP.COM

Patient ID: 113538

DOB: 08/12/1978

Age: 45 years

Gender: Male

Initial compliance period: 07/19/2023 - 08/17/2023

Compliance met: Yes

Compliance percentage: 100%

Payor: Medicare

# Usage

Usage period: 07/19/2023 - 08/17/2023

|Usage days|30/30 days (100%)|
|---|---|
|&gt;= 4 hours|30 days (100%)|
|&lt; 4 hours|0 days (0%)|
|Usage hours|323 hours 21 minutes|
|Average usage (total days)|10 hours 47 minutes|
|Average usage (days used)|10 hours 47 minutes|
|Median usage (days used)|11 hours 14 minutes|
|Total used hours (value since last reset - 08/17/2023)|931 hours|

# Device Information

Device: AirSense 11 AutoSet

Serial number: 22231038074

Mode: CPAP

Set pressure: 15 cmH2O

EPR: Fulltime

EPR level: 3

# Therapy

|Leaks - L/min|Median: 120.0|95th percentile: 120.0|Maximum: 120.0|
|---|---|---|---|
|Events per

# Finally, Pypdf which also used in langchain


In [10]:
def extract_text_pypdf(pdf_file):
    from pypdf import PdfReader

    reader = PdfReader(pdf_file)
    number_of_pages = len(reader.pages)
    page = reader.pages[0]
    text = page.extract_text()
    return text

text = extract_text_pypdf(pdf_files[1])


In [11]:
print(text)

ABREU, JUAN
07/19/2023 - 08/17/2023
Patient ID: 113538
DOB: 08/12/1978
Age: 45 years
Gender: MaleFORT LAUDERDALE COMPL6600 NW 12TH AVE STE 217FORT LAUDERDALEFlorida, 33309
Phone: 954-772-5052Email: LSOLOGUREN@AHCEQUIP.COM
Compliance Report
Initial compliance period 07/19/2023 - 08/17/2023
Compliance met Yes
Compliance percentage 100%
Payor Medicare
Usage 07/19/2023 - 08/17/2023
Usage days 30/30 days (100%)
>= 4 hours 30 days (100%)
< 4 hours 0 days (0%)
Usage hours 323 hours 21 minutes
Average usage (total days) 10 hours 47 minutes
Average usage (days used) 10 hours 47 minutes
Median usage (days used) 11 hours 14 minutes
Total used hours (value since last reset - 08/17/2023) 931 hours
AirSense 11 AutoSet
Serial number 22231038074
Mode CPAP
Set pressure 15 cmH2O
EPR Fulltime
EPR level 3
Therapy
Leaks - L/min Median:120.095th percentile:120.0Maximum:120.0
Events per hour AI:8.7 HI:0.4AHI:9.1
Apnea Index Central:0.0Obstructive:0.0Unknown:8.6
RERA Index 1.0
Cheyne-Stokes respiration (avera

# Next, let's try pytesseract

In [12]:
def extract_text_ocr(pdf_file):
    from pdf2image import convert_from_path
    import pytesseract
    images = convert_from_path(pdf_file, dpi=300)
    print(f"Converted PDF to {len(images)} images")
    text = ''
    for image in images:
        text += pytesseract.image_to_string(image) + "\n"
    return text

text = extract_text_ocr(pdf_files[7])   
print([text])


Converted PDF to 4 images
['SPLIT (NIGHT FRREPORT MUBISQONS STF Dl SOSTTAIRERG ENE BIRER\n\nNanna: StudyDBiate?/22/2B)32023\n\nDOB: Gender: Male Medira IRRedE\n\nHeight: 5"11\'1" Weigfit: 2220/t@ lbs BMI: 3088s Sleep SSpeiciaiis\n\nPhysician): ReiseceddoffraanD DO Scorimyl Tet h0gdanedves RPSGT\nType oof Tas G@PEALNIGMGHT\n\nChief CGomiplain$n orig apayne ated\n\nMedicatiosscimmanton mikktithist|4ishfish soily sitatvasketinppirilh hydroontnetiniaziglejocsannineaspinirB 18h ang,\niprattoopionbrdonoamide\n\ni ; ogra DGOjEN sleepforosisix monaore\nhowe wit aebysicion igexiew raed dhterpret atin Afbilowiymemanatercs wane: costck 3-2, FaLbM 1 Aa oe AM DJORIMA, OL Ma) BOC, Ml,\nLOC-M2, GhimE EMGR RA LAT hanasphessu ce tkenmistosnoressensoRiP (chest, abdtonten) pullseooniatetryodpadyitiposiAdinsaoniggofollews\nthe reno mencatiationéittre AAmerncahcatcade nt 5 of Sléep cid iinessAH I (Apa ai yalypoqarsdbnsite )isifveheumbenbé aporap remathipyqojeaeas\nwith aat40/ciecdetatuicatiper hoounfafiste

# Time and Quality Analsis 

For the time comparison, let's run our script which calls all these functions for all our files and measures the time taken

In [13]:
# Let's create a function to call all these functions and measure the time taken
import time

def calculate_time(pdf_file, function):
    print(f"Running {function.__name__}")
    start_time = time.time()
    text = function(pdf_file)
    end_time = time.time()
    print(f"Time taken: {end_time - start_time} seconds")
    return text, end_time - start_time


In [14]:
list_of_functions = [extract_text_pymupdf4llm, extract_text_pymupdf, extract_text_pdfminer, extract_text_llama_index, extract_text_pypdf]
# list_of_functions = [extract_text_ocr]
results = {}

for function in list_of_functions:
    function_name = function.__name__
    results[function_name] = {
        'total_time': 0,
        'per_file_times': {},
        'texts': {}
    }
    
    for pdf_file in pdf_files:
        text, time_taken = calculate_time(pdf_file, function)
        file_name = pdf_file.split('/')[-1]  # Get just the filename
        
        results[function_name]['per_file_times'][file_name] = time_taken
        results[function_name]['total_time'] += time_taken
        results[function_name]['texts'][file_name] = text


Running extract_text_pymupdf4llm
Processing /Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Compliance Report 4.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Time taken: 0.18959832191467285 seconds
Running extract_text_pymupdf4llm
Processing /Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Compliance Report 1.pdf...
[                                        ] (0/4=========[==========                              ] (1/4=========[====================                    ] (2/4=========[==============================          ] (3/4=========[========================================] (4/4]
Time taken: 1.8267688751220703 seconds
Running extract_text_pymupdf4llm
Processing /Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Compliance Report 2.pdf...
[                                        ] (0/5=======[========                                ] (1/5=======[=====

In [26]:
# # Save results to JSON
# import json
# with open('extraction_results.json', 'w', encoding='utf-8') as f:
#     json.dump(results, f, ensure_ascii=False, indent=4)


# Let's load the results and print the summary

In [27]:
# Load the results
import json
with open('extraction_results.json', 'r', encoding='utf-8') as f:
    results = json.load(f)

# Print summary of total times
for function_name, data in results.items():
    print(f"\n{function_name} total time: {data['total_time']:.2f} seconds")




extract_text_pymupdf4llm total time: 19.45 seconds

extract_text_pymupdf total time: 0.08 seconds

extract_text_pdfminer total time: 2.19 seconds

extract_text_llama_index total time: 654.60 seconds

extract_text_pypdf total time: 0.34 seconds

extract_text_ocr total time: 241.48 seconds


In [28]:
results.keys()

dict_keys(['extract_text_pymupdf4llm', 'extract_text_pymupdf', 'extract_text_pdfminer', 'extract_text_llama_index', 'extract_text_pypdf', 'extract_text_ocr'])

In [39]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot figure with 5 rows
fig = make_subplots(rows=5, cols=1, subplot_titles=list(results['extract_text_pymupdf4llm']['texts'].keys())[:5],
                    vertical_spacing=0.1)

# Color scale for the bars
colors = px.colors.qualitative.Set3

for i, file_name in enumerate(list(results['extract_text_pymupdf4llm']['texts'].keys())[:5], start=1):
    word_counts = {function_name: len(data['texts'][file_name].split()) 
                  for function_name, data in results.items()}
    
    # Add bar trace for each file
    fig.add_trace(
        go.Bar(
            x=list(word_counts.values()),
            y=list(word_counts.keys()),
            orientation='h',
            marker_color=colors[i-1],
            name=file_name,
            text=list(word_counts.values()),  # Add text labels
            textposition='auto',
        ),
        row=i, col=1
    )

# Update layout
fig.update_layout(
    height=1200,  # Increase height to accommodate all subplots
    showlegend=False,
    title_text="Word Count Comparison Across Different Methods",
    title_x=0.5,  # Center the title
    font=dict(size=10),
)

# Update axes labels
fig.update_xaxes(title_text="Word Count")
fig.update_yaxes(title_text="Method")

# Add hover template
fig.update_traces(
    hovertemplate="<b>Method:</b> %{y}<br>" +
                  "<b>Word Count:</b> %{x}<br>" +
                  "<extra></extra>"
)

fig.show()


# So we see that llama_index and pytessract are the best performing method

However, we see the llama-parse also extracts the hidden text in the pdf which is not visible in the original pdf. So I'll be using pytesseract for the final results.


In [40]:
# Replace the text in the results json creating a final json file

final_results = results['extract_text_ocr']

import json
with open('selected_results.json', 'w', encoding='utf-8') as f:
    json.dump(final_results, f, ensure_ascii=False, indent=4)

### Special Case : Skewed Image : Sleep Report 4

In [41]:
pdf_files[7]

'/Users/deveshsurve/UNIVERSITY/PROJECT/classify-pdf/data_files/Sleep Study Report 4.pdf'

### Deskew and Enhance Contrast

Here we will use PyMuPDF and pdf2image to extract the images and then use cv2 to deskew and enhance the contrast

In [42]:
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import cv2
import numpy as np
from PIL import Image

# Step 1: Extract PDF as Images
pdf_path = pdf_files[7]
images = convert_from_path(pdf_path, dpi=300)

# Step 2: Deskew and Enhance Contrast
def deskew_image(image):
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    coords = np.column_stack(np.where(edges > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = gray.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    deskewed = cv2.warpAffine(gray, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return deskewed

def enhance_contrast(image):
    _, thresholded = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresholded

processed_images = [deskew_image(img) for img in images]

# Step 3: Save Back as PDF
pil_images = [Image.fromarray(img) for img in processed_images]
pil_images[0].save(folder_path + "/" + "Sleep Study Report 4 Deskew.pdf", save_all=True, append_images=pil_images[1:])


# Future Work : 
1. Experiment with Markdown and Text
2. Experiment with different OCR libraries like paddlepaddle, easyocr, etc.
3. Add CV functions to the pipeline like deskew, enhance contrast, etc and compare the results